In [39]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from datetime import datetime
from torchvision import datasets, transforms
from torch.utils.data import random_split
from matplotlib import pyplot as plt

torch.manual_seed(123)

In [40]:
# Based on code from the pytorch tutorials

data_path = '../data/'
bp_filter = lambda x: x[1] in [0,2] # Bird and Plane Filter. Or if you will, Everything but Bird and Plane Filter
preprocessor = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.4915, 0.4823, 0.4468),
                                (0.2470, 0.2435, 0.2616))
        ])

cifar10_train_val = list(filter(bp_filter, datasets.CIFAR10(data_path, train=True, download=True, transform=preprocessor)))
cifar10_test = list(filter(bp_filter, datasets.CIFAR10(data_path, train=False, download=True, transform=preprocessor)))

label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']

# For each dataset, keep only airplanes and birds
cifar2_train_val = [(img, label_map[label]) for img, label in cifar10_train_val if label in [0, 2]]
cifar2_test = [(img, label_map[label]) for img, label in cifar10_test if label in [0, 2]]

n_train = int(len(cifar2_train_val)*0.90)
n_val =  len(cifar2_train_val) - n_train
cifar2_train, cifar2_val = random_split(
        cifar2_train_val, 
        [n_train, n_val],
        generator=torch.Generator().manual_seed(123)
)


Files already downloaded and verified
Files already downloaded and verified


In [41]:
cifar2_test[500]

(tensor([[[-0.8785, -1.4659, -1.3389,  ..., -1.2278, -1.3707, -0.5133],
          [-0.9896, -1.6565, -1.4977,  ..., -1.3072, -1.4501, -0.6404],
          [-0.8309, -1.3866, -1.2754,  ..., -1.1325, -1.3707, -0.4816],
          ...,
          [-0.8626, -1.4501, -1.3072,  ..., -1.4501, -1.5612, -0.6086],
          [-1.0214, -1.5930, -1.5453,  ..., -1.4818, -1.5771, -0.6086],
          [-0.8309, -1.3866, -1.3389,  ..., -1.3866, -1.4183, -0.4975]],
 
         [[-0.8533, -1.4492, -1.3204,  ..., -1.1754, -1.3365, -0.4829],
          [-0.9661, -1.6425, -1.4814,  ..., -1.3204, -1.4331, -0.6118],
          [-0.7889, -1.3687, -1.2560,  ..., -1.1916, -1.3687, -0.4507],
          ...,
          [-0.8372, -1.4170, -1.2882,  ..., -1.4331, -1.5459, -0.5796],
          [-0.9983, -1.5781, -1.5298,  ..., -1.4653, -1.5620, -0.5796],
          [-0.8050, -1.3687, -1.3204,  ..., -1.3687, -1.4009, -0.4668]],
 
         [[-0.6586, -1.2133, -1.0933,  ..., -1.1533, -1.1833, -0.3138],
          [-0.7635, -1.3931,

In [42]:
print("Size of the train dataset:        ", len(cifar2_train))
print("Size of the validation dataset:   ", len(cifar2_val))
print("Size of the test dataset:         ", len(cifar2_test))

Size of the train dataset:         9000
Size of the validation dataset:    1000
Size of the test dataset:          2000


In [43]:
class MyMLP(nn.Module):
    def __init__(self):
        super().__init__()  # to inherit the '__init__' method from the 'nn.Module' class
        # Add whatever you want here (e.g layers and activation functions)
        # The order and names don't matter here but it is easier to understand
        # if you go for Layer1, fun1, layer2, fun2, etc
        # Some conventions:
        # - conv stands for convolution
        # - pool for pooling
        # - fc for fully connected

        self.flat = nn.Flatten()
        # 32*32*3: determined by our dataset: 32x32 RGB images
        self.fc1 = nn.Linear(32*32*3, 512)
        self.fc2 = nn.Linear(512, 128)
        self.act2 = nn.ReLU()
        # 2: determined by our number of classes (birds and planes)
        self.fc3 = nn.Linear(128, 32)
        self.fc4 = nn.Linear(32, 2)
        
    # Remember, we saw earlier that `forward` defines the 
    # computation performed at every call (the forward pass) and that it
    # should be overridden by all subclasses.
    def forward(self, x):
        # Now the order matters! 
        out = self.flat(x)
        out = self.act2(self.fc1(out))
        out = self.act2(self.fc2(out))
        out = self.act2(self.fc3(out))
        out = self.fc4(out)
        return out
        

## Training

In [44]:
def train(n_epochs, optimizer, model, loss_fn, train_loader):
    
    n_batch = len(train_loader)
    
    # We'll store there the training loss for each epoch
    losses_train = []
    
    # Set the network in training mode
    model.train()
    
    # Re-initialize gradients, just in case the model has been inappropriately 
    # manipulated before the training
    optimizer.zero_grad(set_to_none=True)
    
    for epoch in range(1, n_epochs + 1): 
        
        # Training loss for the current epoch
        loss_train = 0

        # Loop over our dataset (in batches the data loader creates for us)
        for imgs, labels in train_loader:
            
            # Feed a batch into our model
            outputs = model(imgs)
            
            # Compute the loss we wish to minimize 
            # Note that by default, it is the mean loss that is computed
            # (so entire_batch_loss / batch_size)
            loss = loss_fn(outputs, labels) 
            #print(loss)
            
            # Perform the backward step. That is, compute the gradients of all parameters we want the network to learn
            loss.backward()
            
            # Update the model
            optimizer.step() 
            
            # Zero out gradients before the next round (or the end of training)
            optimizer.zero_grad() 

            # Update loss for this epoch
            # It is important to transform the loss to a number with .item()
            loss_train += loss.item()
            
        # Store current epoch loss. 
        losses_train.append(loss_train / n_batch)
        

        if epoch == 1 or epoch % 10 == 0:
            print('{}  |  Epoch {}  |  Training loss {:.3f}'.format(
                datetime.now().time(), epoch, loss_train / n_batch))
            
    return losses_train

In [45]:
def train_manual_update(n_epochs, model, loss_fn, train_loader, lr=0.01, weight_decay=0, momentum=0):
    n_batch = len(train_loader)
    model.train()
    loss_train = []
    vel = []
    for epoch in range(1, n_epochs + 1):
        losses = 0
        for img, label in train_loader:
            output = model(img)
            loss = loss_fn(output, label)
            
            losses += loss #loss ble ikke plusset   
            loss.backward()
    
            with torch.no_grad():
                for i, p in enumerate(model.parameters()):
                    if epoch == 1:
                        vel.append(torch.zeros(p.data.size(), dtype=torch.float).to(device="cpu"))
                    
                    grad = p.grad
                    
                    if weight_decay > 0:
                        grad = grad + weight_decay*p.data
                        
                    if momentum > 0:
                        vel[i] = momentum * vel[i] + grad
                        tempy = lr*(weight_decay*p.data) #nb
                        p.data = p.data - lr*vel[i] - tempy
                        
                    else:
                        tempy = lr*(weight_decay*p.data)
                        p.data = p.data - (lr*grad) - tempy
                        
                    p.grad.data.zero_()
        loss_train.append(losses/n_batch)
        if epoch == 1 or epoch % 10 == 0:
            print('{}  |  Epoch {}  |  Training loss {:.3f}'.format(
                datetime.now().time(), epoch, losses / n_batch))
    return loss_train

        

In [46]:
train_loader = torch.utils.data.DataLoader(cifar2_train, batch_size=64, shuffle=True)

In [47]:
torch.manual_seed(123)
model = MyMLP()

optimizer = optim.SGD(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

loss_train = train(3, optimizer, model, loss_fn, train_loader)
loss_train



14:12:35.934683  |  Epoch 1  |  Training loss 0.640


[0.6401891835192417, 0.5386637138559464, 0.4816735302302854]

In [48]:

torch.manual_seed(123)

model_manual = MyMLP()

loss_fn = nn.CrossEntropyLoss()

loss_train = train_manual_update(10, model_manual, loss_fn, train_loader)
loss_train


14:17:25.151427  |  Epoch 1  |  Training loss 0.640
14:18:03.754331  |  Epoch 10  |  Training loss 0.332


[tensor(0.6402, grad_fn=<DivBackward0>),
 tensor(0.5387, grad_fn=<DivBackward0>),
 tensor(0.4817, grad_fn=<DivBackward0>),
 tensor(0.4505, grad_fn=<DivBackward0>),
 tensor(0.4247, grad_fn=<DivBackward0>),
 tensor(0.4016, grad_fn=<DivBackward0>),
 tensor(0.3825, grad_fn=<DivBackward0>),
 tensor(0.3638, grad_fn=<DivBackward0>),
 tensor(0.3477, grad_fn=<DivBackward0>),
 tensor(0.3318, grad_fn=<DivBackward0>)]

In [49]:
model01 = MyMLP()
model02 = MyMLP()
model03 = MyMLP()
model04 = MyMLP()
models = [model01, model02, model03, model04]
hyperparams = [[0.1, 0.9, 0], [0.1, 0.9, 0.01], [0.1, 0.9, 0.001], [0.1, 0.8, 0.01]]

train_loader = torch.utils.data.DataLoader(cifar2_train, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64, shuffle=False)


In [51]:
for model, hpars in zip(models, hyperparams):
    print(f"lr, mom, decay: {hpars}")
    train_manual_update(20, model, loss_fn, train_loader, hpars[0], hpars[2], hpars[1])
    print()

lr, mom, decay: [0.1, 0.9, 0]
15:02:07.207045  |  Epoch 1  |  Training loss 0.695
15:02:45.944580  |  Epoch 10  |  Training loss 0.695
15:03:23.002585  |  Epoch 20  |  Training loss 0.695

lr, mom, decay: [0.1, 0.9, 0.01]
15:03:27.832577  |  Epoch 1  |  Training loss 0.519
15:04:04.996136  |  Epoch 10  |  Training loss 0.519
15:04:51.012466  |  Epoch 20  |  Training loss 0.611

lr, mom, decay: [0.1, 0.9, 0.001]
15:04:55.373120  |  Epoch 1  |  Training loss 0.525
15:05:31.481942  |  Epoch 10  |  Training loss 0.436
15:06:11.574315  |  Epoch 20  |  Training loss 0.695

lr, mom, decay: [0.1, 0.8, 0.01]
15:06:15.927678  |  Epoch 1  |  Training loss 0.505
15:06:53.035188  |  Epoch 10  |  Training loss 0.432
15:07:35.289319  |  Epoch 20  |  Training loss 0.428



## Evaluation

In [52]:
def compute_accuracy(model, loader):
    model.eval()
    correct = 0
    total = 0

    # We do not want gradients here, as we will not want to update the parameters.
    with torch.no_grad():
        for imgs, labels in loader:

            outputs = model(imgs)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.shape[0]
            correct += int((predicted == labels).sum())

    acc =  correct / total
    print("Accuracy: {:.2f}".format(acc))
    return acc

In [53]:
accs = []
for model, hpars in zip(models, hyperparams):
    print(f"\nlr, mom, decay: {hpars}")
    print("Training accuracy:")
    compute_accuracy(model, train_loader)
    print("Validation accuracy:")
    acc = compute_accuracy(model, val_loader)
    accs.append(acc)


best_acc = accs.index(max(accs))
best_model = models[best_acc]
print(f"\nlr, mom, decay of best model: {hyperparams[best_acc]}", "\n", best_model)


lr, mom, decay: [0.1, 0.9, 0]
Training accuracy:
Accuracy: 0.50
Validation accuracy:
Accuracy: 0.50

lr, mom, decay: [0.1, 0.9, 0.01]
Training accuracy:
Accuracy: 0.75
Validation accuracy:
Accuracy: 0.78

lr, mom, decay: [0.1, 0.9, 0.001]
Training accuracy:
Accuracy: 0.50
Validation accuracy:
Accuracy: 0.50

lr, mom, decay: [0.1, 0.8, 0.01]
Training accuracy:
Accuracy: 0.82
Validation accuracy:
Accuracy: 0.80

lr, mom, decay of best model: [0.1, 0.8, 0.01] 
 MyMLP(
  (flat): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=3072, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (act2): ReLU()
  (fc3): Linear(in_features=128, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=2, bias=True)
)
